<a href="https://colab.research.google.com/github/mohamedyosef101/101_learning_area/blob/area/Time%20Series/Practical%20Time%20Series%20Analysis%20-%20Nielsen/0-data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Data Collection

---
**Source:** Aileen Nielsen. [Practical Time Series Analysis](https://www.oreilly.com/library/view/practical-time-series/9781492041641/), 2019.O'Reilly.

---
**Scenario:**
Imagine working for a large nonprofit organization. You have been tracking a variety of factors:
* An email recipient's reaction to emails over time: Did they open the emails or not?
* A membership history: Were there periods when a member let their membership lapse?
* Transaction history: When does an individual buy and can we predict this?



In [1]:
# Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Get the data source (from the book repo)
PATH = 'https://github.com/PracticalTimeSeriesAnalysis/BookRepo/raw/master/Ch02/data/'

# Read the datasets
year_joined = pd.read_csv(PATH + 'year_joined.csv')
emails = pd.read_csv(PATH + 'emails.csv')
donations = pd.read_csv(PATH + 'donations.csv')

As you see, we have serveral related datasets available. So, we will need to line them up together, possibly dealing with disparate timestamping conventions or different leveles of granularity in the data.

- `year_joined` The year each member joined and currend status of membership.
- `emails` Number of emails you sent out in a given week that were opened by the member.
- `donations` Time a member donated to your organization.

## Step 1. **Examining the data**

You should start by considering the temporal axes of the data we have. In the preceding tables we have three levels of temporal resolution:
* A yearly member status Retrofitting a Time Series
* A weekly tally of emails opened
* Instantaneous timestamps of donations

In [2]:
year_joined.yearJoined.value_counts()

yearJoined
2018    362
2017    291
2016    159
2015    102
2014     86
Name: count, dtype: int64

In [3]:
emails.week.count()

25488

In [4]:
donations.timestamp.min(), donations.timestamp.max()

('2015-02-10 13:05:21', '2018-06-03 21:05:19')

In [5]:
donations.timestamp.count()

2676

You will also need to examine whether the data means what you think it means. For example, you would want to determine whether the member status is a yearly status or just the most recent status. One way to answer this is to check whether any member has more than one entry:

In [6]:
year_joined.groupby('user').count().value_counts()

userStats  yearJoined
1          1             1000
Name: count, dtype: int64

Here we can see that all 1000 users have only one status, so that the year they joined is indeed likely to be the year joined, accompanied by a status that may be the user's current status or status when they joined.

> **Note:** If you were applying a member's current status to an analysis of past data, that would be a lookahead because you would be inputting something into a time series model that could not be known at the time. This is why you would not want to use a status variable, such as YearJoined, without knowing when it was assigned.

In [7]:
emails.head()

,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00


Looking at the emails table, it is obvious that the data is a weekly timestamp. You could start by asking how the weeks are reported in time. While we may not have information to restructure the table, if the week is divided in a somewhat strange way relative to our industry, we might want to know about this too. For analyzing human activities, it generally makes sense to look at the calendar week of Sunday through Saturday or Monday through Sunday rather than weeks that are less in line with the cycle of human activity. So, for example, don't arbitrarily start your week with January 1st.

In [9]:
emails['week'] = pd.to_datetime(emails.week)
emails['day'] = emails.week.dt.day_name()
emails.head()

,emailsOpened,user,week,day
0,3.0,1.0,2015-06-29,Monday
1,2.0,1.0,2015-07-13,Monday
2,2.0,1.0,2015-07-20,Monday
3,3.0,1.0,2015-07-27,Monday
4,1.0,1.0,2015-08-03,Monday


## Step 2. **Data cleaning**

You could also ask whether null weeks are reported? That is, do the weeks in which the member opened 0 emails have a place in the table? This matters when we want to do time-oriented modeling. In such cases we need to always have the null weeks present in the data because a 0 week is still a data point.

In [10]:
emails[emails.emailsOpened < 1].emailsOpened.count()

0

There are two possibilities: either nulls are not reported or members always have at least one email event. Anyone who has worked with email data knows that it's difficult to get people to open emails, so the hypothesis that members always open at least one email per week is quite unlikely. In this case, we can resolve this by looking at the history of just one user:

In [11]:
emails[emails.user == 998].head()

,emailsOpened,user,week,day
25464,1.0,998.0,2017-12-04,Monday
25465,3.0,998.0,2017-12-11,Monday
25466,3.0,998.0,2017-12-18,Monday
25467,3.0,998.0,2018-01-01,Monday
25468,3.0,998.0,2018-01-08,Monday


We can see that some weeks are missing. There aren't any Dec 2017 email events after Dec 04, 2017. Below is the code for that.

We can check this more mathematically by calculating how many weekly observations we should have between the first and last event for that member. First we calculate the length of the member's tenure, in weeks:

In [12]:
(emails[emails.user == 998].week.max() -
  emails[emails.user == 998].week.min()).days/7 + 1
# I've added 1 to account for the offset at the end

26.0

Then we see how many weeks of data we have for that member:

In [13]:
emails[emails.user == 998].shape

(24, 4)

It’s a lot easier to fill in all missing weeks for all members by exploiting Pandas’
indexing functionality, rather than writing our own solution. We can generate a
MultiIndex for a Pandas data frame, which will create all combinations of weeks and
members—that is, a Cartesian product:

In [14]:
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))
complete_idx

MultiIndex([('2016-11-07',   1.0),
            ('2016-11-07',   3.0),
            ('2016-11-07',   5.0),
            ('2016-11-07',   6.0),
            ('2016-11-07',   9.0),
            ('2016-11-07',  10.0),
            ('2016-11-07',  14.0),
            ('2016-11-07',  16.0),
            ('2016-11-07',  20.0),
            ('2016-11-07',  21.0),
            ...
            ('2017-07-03', 973.0),
            ('2017-07-03', 977.0),
            ('2017-07-03', 982.0),
            ('2017-07-03', 984.0),
            ('2017-07-03', 987.0),
            ('2017-07-03', 991.0),
            ('2017-07-03', 992.0),
            ('2017-07-03', 993.0),
            ('2017-07-03', 995.0),
            ('2017-07-03', 998.0)],
           length=93247)

In [16]:
# Also, I do not need the day column anymore
emails.drop('day', axis=1, inplace=True)

We use this index to reindex the original table and fill in the missing values--inthis case with 0 on the assumption that nothing recoreded means there was nothing to record. We also reset the index to make the user and week information available as columns, and name those columns:

In [17]:
all_email = emails.set_index(['week', 'user']).reindex(complete_idx,
                                                       fill_value=0).reset_index()
all_email.columns = ['week', 'user', 'email_opened']

In [23]:
all_email[all_email.email_opened == 0].email_opened.count()

67759

Notice that we have a large number of zeros at the start. These are likely before the member joined the organization, so they would not have been on an email list. There are not too many kinds of analyses where we'd want to keep the member's truly null weeks around—specifically those weeks before the member ever indicated opening an email. If we had the precise date a user started receiving emails, we would have an objective objective cutoff. As it is, we will let the data guide us. For each member we determine the `start_date` and `end_date` cutoffs by grouping the email `DataFrame` per user and selecting the maximum and minimum week values:

In [24]:
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()

,user,min,max
0,1.0,2015-06-29,2018-05-28
1,3.0,2018-03-05,2018-04-23
2,5.0,2017-06-05,2018-05-28
3,6.0,2016-12-05,2018-05-28
4,9.0,2016-07-18,2018-05-28
...,...,...,...
534,991.0,2016-10-24,2016-10-24
535,992.0,2015-02-09,2015-07-06
536,993.0,2017-09-11,2018-05-28
537,995.0,2016-09-05,2018-05-28


We drop rows from the DataFrame that don't contribute sensibly to the chronology, specifically 0 rows before each member's first nonzero count:

In [ ]:
%%capture
for _, row in cutoff_dates.iterrows():
  user = row['user']
  start_date = row['min']
  end_date = row['max']
  all_email.drop(
      all_email[all_email.user == user][all_email.week < start_date].index,
      inplace=True
  )
  all_email.drop(
      all_email[all_email.user == user][all_email.week > end_date].index,
      inplace=True
      )